Need to have the following files in the same folder:

REQUIREMENTS.TXT

EXAMPLE_MD_TO_TEXT.TXT

example.pdf

HF_TOKEN.txt

GROQ_TOKEN.txt


In [1]:
with open("example_md_to_text.txt", "r", encoding="latin-1") as f: # TO DO: check proper encoding for .md files
    markdown_example = f.read()

with open("example.pdf", "rb") as f:
    pdf = f.read()

with open("HF_TOKEN.txt", "r") as f:
    hf_token = f.read()

with open("GROQ_KEY.txt", "r") as f:
    groq_token = f.read()

In [2]:
from pprint import pprint
import tqdm
# pprint(markdown_example)

Basic OLlama Workflow (unused)

In [3]:
# import ollama

In [4]:
# # basic workflow of getting llama embeddings with ollama
# ollama.embeddings(model="llama3.2:3b",
#                 prompt="Hello")

In [5]:
# # basic workflow of generating responses with ollama
# ollama.generate(model="llama3.2:3b",
#                 prompt="Whos is Obama")

Loading HF to get Attention Masks

In [6]:
from huggingface_hub import HfFolder, whoami

HfFolder.save_token(hf_token)
print(whoami()["name"])

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


alberto-lorente


In [7]:
import torch

device = "cpu"
if torch.cuda.is_available():
  print("Cuda available")
  device = torch.device('cuda')

Cuda available


In [8]:
# using huggingface tokenizer for attettion layer
# make sure you have GPU enabled
# takes around 5 mins to load with TPUs

from transformers import AutoTokenizer, AutoModel

model_id = "HIT-TMG/KaLM-embedding-multilingual-mini-v1" # which model to use?
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id)
model = AutoModel.from_pretrained(model_id).to(device)

In [9]:
model

Qwen2Model(
  (embed_tokens): Embedding(151936, 896)
  (layers): ModuleList(
    (0-23): 24 x Qwen2DecoderLayer(
      (self_attn): Qwen2SdpaAttention(
        (q_proj): Linear(in_features=896, out_features=896, bias=True)
        (k_proj): Linear(in_features=896, out_features=128, bias=True)
        (v_proj): Linear(in_features=896, out_features=128, bias=True)
        (o_proj): Linear(in_features=896, out_features=896, bias=False)
        (rotary_emb): Qwen2RotaryEmbedding()
      )
      (mlp): Qwen2MLP(
        (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
        (up_proj): Linear(in_features=896, out_features=4864, bias=False)
        (down_proj): Linear(in_features=4864, out_features=896, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
    )
  )
  (norm): Qwen2RMSNorm((896,), eps=1e-06)
  (rotary_emb): Qwen2RotaryEmbedding()
)

## Splitting the text into sentences and paragraphs

In [10]:
import spacy

nlp = spacy.load("fr_core_news_sm")

In [11]:
doc = nlp(markdown_example)
sents = [sent.text for sent in doc.sents]

In [12]:
sents[0:10]

['#',
 'Séance publique\n\n\n\n',
 '# du jeudi 28 octobre 2021\n\n\n\n',
 '# à 18h00\n\n\n\n#',
 'Chorum',
 'Alain Gilles',
 '- Halle Vacheresse\n\n\n\n',
 '#',
 'Rue des Vernes à Roanne\n\n\n\n',
 '#']

In [13]:
import numpy as np

# to reconstruct based on sentences, we can iterate through the list with a range and
# concat every 10 sents into a single string

rang_sentence_union = np.arange(start=0, stop=len(sents), step=10)
# print(rang)
print("Total number of sents: ", len(sents))
print("Number of final chunks: ", len(rang_sentence_union))


Total number of sents:  905
Number of final chunks:  91


In [14]:
paragraphs = []
i = 0

# merging sentences based on the ranges
while i+1 < len(rang_sentence_union):
  start = rang_sentence_union[i]
  stop = rang_sentence_union[i+1]
  # print(start, stop)

  subset_to_join = sents[start : stop]
  sent_union = " ".join(subset_to_join)

  paragraph_info = {"paragraph_union": sent_union,
                    "start_range": start,
                    "stop_range": stop,
                    "list_sents":subset_to_join}

  paragraphs.append(paragraph_info)

  i += 1

# if the stop of the range comes before the last sentence, we take those final couple of sentences
# and add them to the last sentence of the paragraph list

if stop != len(sents):

  subset_to_join = sents[stop : len(sents)]
  final_sents = " ".join(subset_to_join)
  para_to_edit = paragraphs.pop(-1)
  final_union = para_to_edit["paragraph_union"] + " " + final_sents

  para_to_edit["paragraph_union"] =  final_union
  para_to_edit["stop_range"] = len(sents)
  para_to_edit["list_sents"].extend(subset_to_join)

  paragraphs.append(para_to_edit)

In [15]:
paragraphs[-1]

{'paragraph_union': 'Ajustement de la contribution au titre des eaux pluviales pour l\x92opération Foch Sully (-400 k\x80), des contributions au SEEDR pour le traitement des ordures ménagères (-189,84 k\x80), des participations du budget général aux budgets annexes (48,5 k\x80), de subventions diverses (19,4 k\x80), des créances admises en non-valeur (-12 k\x80) et changement de nature comptable (et de chapitre) à la demande de la trésorerie (57 k\x80).\n\n\n\n # Charges exceptionnelles : 54 139 \x80\n\n\n\nDépenses liées au fonctionnement du Centre de vaccination au Fuyant pour la période du 1er septembre au 31 décembre 2021 (80 k\x80), changement de nature comptable (et de chapitre) à la demande de la trésorerie (-57 k\x80), ajustement des dépenses liées au COVID (-9,78 k\x80), remboursement des abonnements et cours au Nauticum (8 k\x80), reversement du résultat de la commune Les Noës à la Roannaise de l\x92Eau (2,8 k\x80) et de divers dépenses exceptionnelles (30,119 k\x80).\n\n\n\n

In [16]:
def split_markdown_to_paras(text, spacy_model="fr_core_news_sm", n_sents_per_para=10):

    nlp = spacy.load(spacy_model)
    doc = nlp(text)
    sents = [sent.text for sent in doc.sents]

    rang_sentence_union = np.arange(start=0, stop=len(sents), step=n_sents_per_para)
    # print("Total number of sents: ", len(sents))
    # print("Number of final chunks: ", len(rang_sentence_union))


    paragraphs = []
    i = 0

    # merging sentences based on the ranges
    while i+1 < len(rang_sentence_union):
        start = rang_sentence_union[i]
        stop = rang_sentence_union[i+1]
        # print(start, stop)

        subset_to_join = sents[start : stop]
        sent_union = " ".join(subset_to_join)

        paragraph_info = {"paragraph_union": sent_union,
                        "start_range": start,
                        "stop_range": stop,
                        "list_sents":subset_to_join}

        paragraphs.append(paragraph_info)

        i += 1

    # if the stop of the range comes before the last sentence, we take those final couple of sentences
    # and add them to the last sentence of the paragraph list
    if stop != len(sents):

        subset_to_join = sents[stop : len(sents)]
        final_sents = " ".join(subset_to_join)
        para_to_edit = paragraphs.pop(-1)
        final_union = para_to_edit["paragraph_union"] + " " + final_sents

        para_to_edit["paragraph_union"] =  final_union
        para_to_edit["stop_range"] = len(sents)
        para_to_edit["list_sents"].extend(subset_to_join)
        paragraphs.append(para_to_edit)

    return paragraphs

Now we have a list of dictionaries, one for each constructed paragraph. So far, each paragraph has the union string, start and end indexes as well as the list of individual sentences that were joined.

## Getting the text embeddings


NOTE: There is one important caveat. We are computing sentence embeddings. If we were to pass the whole doc, the attention mask for each token would look very different.

### Example for one piece of text

In [17]:
# dir(tokenizer)

In [18]:
tokenized_markdown = tokenizer(markdown_example, return_tensors="pt").to(device)

In [19]:
len(tokenized_markdown[0])

20468

In [20]:
model.eval()
with torch.no_grad():
  # first batch (only one since we are processing one doc)
  # final token
    embeddings = model(**tokenized_markdown)[0][:, 0].squeeze(0)

In [21]:
embeddings.shape

torch.Size([896])

We take the last token since it's the one which contains all the aggregate info?

In [22]:
# embeddings

In [23]:
import torch.nn.functional as F

normalized_embeddings = F.normalize(embeddings, p=2, dim=0)

In [24]:
# normalized_embeddings

### For a list of sentences

In [25]:
def compute_norm_embeddings(tokenizer, model, sentence):

    tokenized_sentences = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True).to(device)

    model.eval()
    with torch.no_grad():
        embeddings = model(**tokenized_sentences)[0][:, 0].squeeze(0) # to take out the unused dimension since we are not batching
        # print(embeddings.shape)

    normalized_embeddings = F.normalize(embeddings, p=2, dim=0)
    detached_embeddings = normalized_embeddings.detach().cpu().numpy() # detached into cpu so that we can manipulate them for clustering

    torch.cuda.empty_cache() # careful with running out of memory

    return detached_embeddings

In [26]:
list_paras = [para["paragraph_union"] for para in paragraphs]

In [27]:
for paras, para_dict in zip(list_paras, paragraphs):
    para_dict["para_embedding"] = compute_norm_embeddings(tokenizer, model, paras)

In [28]:
paragraphs[0]["para_embedding"].shape

(896,)

Now there is an extra key for the embeddings that we will use for the clsutering.

## Clustering

In [29]:
squeezeded_embeddings = [para_dict["para_embedding"] for para_dict in paragraphs]

In [30]:
# squeezeded_embeddings[0:2]

### Example for one number of clusters

In [31]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

In [32]:
gm = GaussianMixture(n_components=4, random_state=42)
clusters = gm.fit_predict(squeezeded_embeddings)

In [33]:
# clusters # label assigned to each sentence

In [34]:
sil_sc = silhouette_score(squeezeded_embeddings, clusters)
print(sil_sc) # the closer to 1 the better (how similar is an object to its cluster compared to the other clusters)

0.44329956


In [35]:
clusters

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 3, 0, 0, 0, 2, 0, 1, 1, 1,
       0, 3, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 3, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 1,
       1, 0])

### Optimal number of clusters

In [36]:
import numpy as np

# initial number of clusters to try
range_clusters = np.arange(start=3, stop=9, step=1)
print(range_clusters)

[3 4 5 6 7 8]


We could compute jensenshannon distance as well.

In [37]:
def cluster_n(cluster_model, n_clusters, embeddings, scoring_function):

  clusters = cluster_model.fit_predict(embeddings)
  sil_sc = scoring_function(embeddings, clusters)

  print("Number of clusters: ", n_clusters)
  print("Score: ", sil_sc)
  print()

  return clusters, sil_sc

In [38]:
silhouette_scores = []
clusters_labels = []
for n_cluster in range_clusters:
  gm = GaussianMixture(n_components=n_cluster, random_state=42)
  clusters, sil_sc = cluster_n(gm, n_cluster, squeezeded_embeddings, silhouette_score)
  silhouette_scores.append(sil_sc)
  clusters_labels.append(clusters)

Number of clusters:  3
Score:  0.44600135

Number of clusters:  4
Score:  0.44329956

Number of clusters:  5
Score:  0.47113436

Number of clusters:  6
Score:  0.48168555

Number of clusters:  7
Score:  0.48469815

Number of clusters:  8
Score:  0.5539949



In [39]:
max = np.argmax(silhouette_scores)
optimal_n = range_clusters[max]
print("Index", max)
print("Optimal Number of Clusters", optimal_n)

Index 5
Optimal Number of Clusters 8


In [40]:
final_clusters = clusters_labels[max]

## Logging Information, concatinating the text relating to each cluster and recomputing embeddings

In [41]:
clusters_ids = {f"cluster_{cluster_id}": {"para_indexes": [],
                                        "union_paras": ""} for cluster_id in np.arange(0, optimal_n, 1)}
clusters_ids

{'cluster_0': {'para_indexes': [], 'union_paras': ''},
 'cluster_1': {'para_indexes': [], 'union_paras': ''},
 'cluster_2': {'para_indexes': [], 'union_paras': ''},
 'cluster_3': {'para_indexes': [], 'union_paras': ''},
 'cluster_4': {'para_indexes': [], 'union_paras': ''},
 'cluster_5': {'para_indexes': [], 'union_paras': ''},
 'cluster_6': {'para_indexes': [], 'union_paras': ''},
 'cluster_7': {'para_indexes': [], 'union_paras': ''}}

In [42]:
# saving the cluster in the paragraphs dictionary
# and the indexes of the paragraphs of cluster in another one

i = 0

for para_dict, cluster in zip(paragraphs, final_clusters):
  cluster_n_string = f"cluster_{cluster}"
  para_dict["para_cluster"] = cluster_n_string
  clusters_ids[cluster_n_string]["para_indexes"].append(i)
  clusters_ids[cluster_n_string]["union_paras"] = clusters_ids[cluster_n_string]["union_paras"] + para_dict["paragraph_union"]
  i = i + 1

In [43]:
# depending on the post-processing techniques, we may not be using these embeddings
# and we'll have to re-tokenize and embed with the model we use

for cluster in clusters_ids.keys():
  text = clusters_ids[cluster]["union_paras"]
  cluster_embds = compute_norm_embeddings(tokenizer, model, text)
  clusters_ids[cluster]["cluster_embedding"] = cluster_embds

In [44]:
# clusters_ids["cluster_0"]
# looking at this example, we can see that the clustering is mostly sequential

In [45]:
paragraphs[0].keys()

dict_keys(['paragraph_union', 'start_range', 'stop_range', 'list_sents', 'para_embedding', 'para_cluster'])

In [46]:
clusters_ids.keys()

dict_keys(['cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6', 'cluster_7'])

In [45]:
clusters_ids["cluster_0"].keys()

dict_keys(['para_indexes', 'union_paras', 'cluster_embedding'])

## Detecting tables and augmenting the context

### Workflow with im2table

In [47]:
markdown_example

"# Séance publique\n\n\n\n# du jeudi 28 octobre 2021\n\n\n\n# à 18h00\n\n\n\n# Chorum Alain Gilles - Halle Vacheresse\n\n\n\n# Rue des Vernes à Roanne\n\n\n\n# PROCES VERBAL\n\n\n\nL'an deux mille vingt et un, le 28 octobre à 18 h 00, les conseillers communautaires de Roannais Agglomération, se sont réunis à l\x92Espace Chorum \x96 Halle Vacheresse \x96 Rue des Vernes à Roanne.\n\n\n\nLa convocation de tous les conseillers a été faite le 22 octobre 2021, dans les formes et délais prescrits par la loi, par Yves Nicolin, Président.\n\n\n\n# Etaient présents :\n\n\n\nChristine Aranéo - Marcel Augier - Jean-Jacques Banchet - Pierre Barnet - Martine Barroso - Isabelle Berthelot - Franck Beysson - Jean-Yves Boire - Romain Bost - Michelle Bouchet - Edmond Bourgeon - Marie-Christine Bravo - Catherine Brun - Dominique Bruyère - Nicolas Chargueros - Jean-Luc Chervin - Christine Chevillard - Pierre Coissard - Patrick Collet - Sandra Creuzet - Hervé Daval - Jean-Paul Descombes - Pierre Devedeux - 

In [48]:
# when downloading pdfs, important to use the datapolitics endpoint
link_pdf = "https://datapolitics-public.s3.gra.io.cloud.ovh.net/LORIA/2289/2e442_PV_CC_2021_10_28.pdf"

In [49]:
import requests

response = requests.get(link_pdf)
response.status_code

200

In [50]:
with open('example.pdf', 'wb') as file:
        file.write(response.content)


In [51]:
from img2table.document import PDF

pdf = PDF('example.pdf',
          pages= [0,1],
          detect_rotation=False,
          pdf_text_extraction=True)

In [52]:
extracted_tables = pdf.extract_tables()

In [57]:
extracted_tables # dictionary of {page_n: [list of tables extracted]}

{0: [],
 1: [ExtractedTable(title=None, bbox=(153, 92, 1498, 1338),shape=(21, 4))]}

In [63]:
print(dir(extracted_tables[1][0]))
# bbox', 'content', 'df', 'html', 'html_repr', 'title
# df returns a pandas df

['__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_to_worksheet', 'bbox', 'content', 'df', 'html', 'html_repr', 'title']


In [70]:
extracted_tables[1][0].html

'<table>\n <tr>\n  <td colspan="1" rowspan="1">\n   Absents\n  </td>\n  <td colspan="1" rowspan="1">\n   Ni pouvoir\n   <br/>\n   Ni suppléant\n  </td>\n  <td colspan="1" rowspan="1">\n   Suppléant\n  </td>\n  <td colspan="1" rowspan="1">\n   Pouvoir donné à…\n  </td>\n </tr>\n <tr>\n  <td colspan="1" rowspan="1">\n   Jean-Marc Ambroise\n  </td>\n  <td colspan="1" rowspan="1">\n   X\n  </td>\n  <td colspan="1" rowspan="1">\n  </td>\n  <td colspan="1" rowspan="1">\n  </td>\n </tr>\n <tr>\n  <td colspan="1" rowspan="1">\n   Laurence Boyer\n  </td>\n  <td colspan="1" rowspan="1">\n  </td>\n  <td colspan="1" rowspan="1">\n   Yves Meunier\n  </td>\n  <td colspan="1" rowspan="1">\n  </td>\n </tr>\n <tr>\n  <td colspan="1" rowspan="1">\n   Marie-France Catheland\n  </td>\n  <td colspan="1" rowspan="1">\n  </td>\n  <td colspan="1" rowspan="1">\n  </td>\n  <td colspan="1" rowspan="1">\n   Muriel Marcellin\n  </td>\n </tr>\n <tr>\n  <td colspan="1" rowspan="1">\n   Yves Chambost\n  </td>\n  <td 

We could augment the context of the table by passing a text reconstruction of the pandas df or we could use the extracted tables return to filter which pages have tables. 

In [71]:
def extract_tables_from_page(pdf_path, list_pages):
    
    pdf = PDF(pdf_path,
                pages= list_pages,
                detect_rotation=False,
                pdf_text_extraction=True)
    
    extracted_tables = pdf.extract_tables()
    return extracted_tables

Works well and can output HTML but it is VERY slow.

### Workflow with ColPali and Byaldi.

Bare in mind ColPali only does image retrieval, it does not have chat support. Works relatively well.

In [72]:
from byaldi import RAGMultiModalModel

# Initialize the model
model = RAGMultiModalModel.from_pretrained("vidore/colpali") # takes a bit to load 

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:34<00:00, 17.38s/it]
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alber\.cache\huggingface\hub\models--vidore--colpali. Caching files w

In [91]:
# import os
# os.listdir()

In [96]:
import base64
from pdf2image import convert_from_path

# converting pdf to image and saving each page
images = convert_from_path('example.pdf', poppler_path=r"poppler-24.08.0\Library\bin") # required to have a poppler release if you are on windows

for i in range(len(images)):
      # Save pages as images in the pdf
    images[i].save('page'+ str(i) +'.jpg', 'JPEG')

In [99]:
def convert_pdf_to_image(pdf_path, output_dir=r"/output_pdf_to_img/", poppler_path=r"poppler-24.08.0\Library\bin"):
    
    images = convert_from_path(pdf_path, poppler_path=poppler_path) 
    for i in range(len(images)):
        # Save pages as images in the pdf
        images[i].save(output_dir + pdf_path + '_page'+ str(i) +'.jpg', 'JPEG')
        
    return images

In [ ]:
# Index documents
model.index(
    input_path="page1.jpg", # input have to be image files! 
    index_name="index_2",
    doc_ids=[2], #the index where it will be stored, has to be free
    store_collection_with_index=False,
    overwrite=True
)

# indexing is very slow

Added page 1 of document 2 to index.
Index exported to .byaldi\index_2
Index exported to .byaldi\index_2


{0: 'page0.jpg', 2: 'page1.jpg'}

In [94]:
# Perform a search
query = "Who was absent of the meeting"
results = model.search(query, k=5)

# Print results
for result in results:
    print(f"Doc ID: {result.doc_id}, Page: {result.page_num}, Score: {result.score}")


Doc ID: 0, Page: 1, Score: 14.0625


In [57]:
# results[0]

Indexing the PDF pages takes a lot of time as well.
A possible solution could be doing an initial query on the paragraph representation and index only those pages that match the query.

### Workflow with Groq and Llama for Vision-Instruction

We could try to get some page number of where the tables are and index/query those pages only.

Challenge: each pdf will produce many images and we'd have to iterate over all of them.

Workaround: filter with Coli/im2pdf, query with Llama.

In [ ]:
# encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# path to your image
image_path = "page1.jpg"

# encoding
base64_image = encode_image(image_path)

In [25]:
from groq import Groq

client = Groq(api_key=groq_token)

text_query = "Who was absent of the meeting?"
messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": text_query},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}", # encoded image
                    },
                },
            ],
        }
    ]

chat_completion = client.chat.completions.create(
    messages=messages,
    model="llama-3.2-11b-vision-preview",
)

print(chat_completion.choices[0].message.content)

Jean-Marc Ambrose was absent of the meeting. 

This information can be found in the table labeled "Absent" with the names of the absentees, where Jean-Marc Ambrose is listed.


Since we will use it to generate textual context for the tables, I will set an original prompt.

Seems to work pretty well!

In [106]:
base_prompt="""CONTEXTE
L'image suivante contient un tableau. 

TÂCHE
Décrivez le tableau en accordant une attention particulière au contexte qui l'entoure, qu'il s'agisse de budgets, de dates, d'élections, d'agendas, de projets futurs ou de sujets connexes. 

FORMAT DE LA RÉPONSE
Votre réponse doit être aussi détaillée que possible. 
Votre résultat doit être la description du tableau directement."""

In [105]:
def format_multimodal_message_list(base_prompt, encoded_image):
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": base_prompt},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encoded_image}", # encoded image
                    },
                },
            ],
        }
    ]
    return messages
    

In [107]:
def augment_multimodal_context(image_path, base_prompt, model="llama-3.2-11b-vision-preview"):
    
    encoded_image = encode_image(image_path)
    messages = format_multimodal_message_list(base_prompt, encoded_image)
    chat_completion = client.chat.completions.create(   messages=messages,
                                                        model=model
                                                        )
    
    return chat_completion.choices[0].message.content

## Generating Summaries

Now that we have the basic separation wrt to clusters, we can start creating different representations for each cluster and doing the hierarchical indexing.

In [ ]:
# https://huggingface.co/mrm8488/camembert2camembert_shared-finetuned-french-summarization

import torch
from transformers import RobertaTokenizerFast, EncoderDecoderModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'

ckpt = 'mrm8488/camembert2camembert_shared-finetuned-french-summarization'
tokenizer = RobertaTokenizerFast.from_pretrained(ckpt)
model = EncoderDecoderModel.from_pretrained(ckpt).to(device)
def generate_summary(text):
    inputs = tokenizer([text], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    output = model.generate(input_ids, attention_mask=attention_mask)
    return tokenizer.decode(output[0], skip_special_tokens=True)
    
text = "Un nuage de fumée juste après l’explosion, le 1er juin 2019. Une déflagration dans une importante usine d’explosifs du centre de la Russie a fait au moins 79 blessés samedi 1er juin. L’explosion a eu lieu dans l’usine Kristall à Dzerzhinsk, une ville située à environ 400 kilomètres à l’est de Moscou, dans la région de Nijni-Novgorod. « Il y a eu une explosion technique dans l’un des ateliers, suivie d’un incendie qui s’est propagé sur une centaine de mètres carrés », a expliqué un porte-parole des services d’urgence. Des images circulant sur les réseaux sociaux montraient un énorme nuage de fumée après l’explosion. Cinq bâtiments de l’usine et près de 180 bâtiments résidentiels ont été endommagés par l’explosion, selon les autorités municipales. Une enquête pour de potentielles violations des normes de sécurité a été ouverte. Fragments de shrapnel Les blessés ont été soignés après avoir été atteints par des fragments issus de l’explosion, a précisé une porte-parole des autorités sanitaires citée par Interfax. « Nous parlons de blessures par shrapnel d’une gravité moyenne et modérée », a-t-elle précisé. Selon des représentants de Kristall, cinq personnes travaillaient dans la zone où s’est produite l’explosion. Elles ont pu être évacuées en sécurité. Les pompiers locaux ont rapporté n’avoir aucune information sur des personnes qui se trouveraient encore dans l’usine."

generate_summary(text)

# Output: L’explosion a eu lieu dans l’usine Kristall à Dzerzhinsk, une ville située à environ 400 kilomètres à l’est de Moscou.


In [ ]:
# https://huggingface.co/plguillou/t5-base-fr-sum-cnndm

from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("plguillou/t5-base-fr-sum-cnndm")
model = T5ForConditionalGeneration.from_pretrained("plguillou/t5-base-fr-sum-cnndm")


In [ ]:
from transformers import AutoModelForSeq2SeqLM

summ_model = 'csebuetnlp/mT5_multilingual_XLSum' # it's too costly

tokenizer_summ = AutoTokenizer.from_pretrained(summ_model)
model_summ = AutoModelForSeq2SeqLM.from_pretrained(summ_model).to(device)

In [ ]:
import re
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip())) # was part of the docs

In [ ]:
example_cluster = clusters_ids["cluster_0"]["union_paras"]
example_cluster

In [ ]:
tokenized_example = tokenizer_summ(
                              [WHITESPACE_HANDLER(example_cluster)],
                              return_tensors="pt",
                              padding="max_length",
                              truncation=True
                          ).to(device)

In [ ]:
tokenized_example

In [ ]:
input_ids = tokenized_example["input_ids"]
# input_ids = tokenized_example["input_ids"].squeeze(dim=0) NOP,expects the unsqueexed dimension

In [ ]:
input_ids

In [ ]:
len(input_ids[0])

In [ ]:
torch.cuda.empty_cache()
output = model_summ.generate(
    input_ids=input_ids,
    max_length=300,
    no_repeat_ngram_size=2,
    num_beams=1) # if it is too slow, adjust num_beams and max_len

In [ ]:
output[0]

In [ ]:
summary = tokenizer_summ.decode(
    output[0],
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

In [ ]:
summary

For clusters

In [ ]:
from torch.amp import autocast

In [ ]:
def generate_summary(concat_text):

  WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

  tokenized_example = tokenizer_summ(
                              [WHITESPACE_HANDLER(concat_text)],
                              return_tensors="pt",
                              padding="max_length",
                              truncation=True).to(device)

  input_ids = tokenized_example["input_ids"]

  model_summ.eval()
  with autocast("cuda"):
    summary_embedding_output = model_summ.generate(
                                              input_ids=input_ids,
                                              max_length=1000,
                                              no_repeat_ngram_size=2,
                                              num_beams=4) # if it is too slow, adjust num_beams and max_len

  summary_decoded = tokenizer_summ.decode(
                                    summary_embedding_output[0],
                                    skip_special_tokens=True,
                                    clean_up_tokenization_spaces=False)

  torch.cuda.empty_cache() # Careful with running out of memory

  return summary_embedding_output.detach().cpu().numpy(), summary_decoded

In [ ]:
torch.cuda.empty_cache()

In [ ]:
def process_cluster(cluster, cluster_aggregates=cluster_aggregates):

  print("Procesing cluster ", cluster+1)
  torch.cuda.empty_cache()

  cluster_aggregates[cluster]["broader_representations"] = {}

  concat_text = " ".join(cluster_aggregates[cluster]["texts"])
  cluster_aggregates[cluster]["broader_representations"]["concat_text"] = concat_text


  cluster_embeddings = compute_norm_embeddings(concat_text)
  cluster_aggregates[cluster]["broader_representations"]["cluster_sentence_embeddings"] = cluster_embeddings[0] # taking out the extra unnecesary dimension

  torch.cuda.empty_cache()

  summary_embedding_output, summary_decoded = generate_summary(concat_text)
  cluster_aggregates[cluster]["broader_representations"]["cluster_summary_embedding"] = summary_embedding_output
  cluster_aggregates[cluster]["broader_representations"]["cluster_summary_decoded"] = summary_decoded

  return cluster_aggregates


In [ ]:
for cluster in list(cluster_aggregates.keys()):
  cluster_aggregates = process_cluster(cluster)

In [ ]:
cluster_aggregates[1]["broader_representations"].keys()

In [ ]:
cluster_aggregates[1]["broader_representations"]["cluster_summary_decoded"]

In [ ]:
pprint(cluster_aggregates[1]["broader_representations"]["concat_text"])

In [ ]:
cluster_aggregates[0]["broader_representations"]["cluster_summary_decoded"]

## Using Groq

In [ ]:
from groq import Groq

client = Groq(api_key=groq_token)


In [ ]:
text_to_sum = "bla bla bla bla bla bla"

summary_prompt = """Il s'agit d'un texte concernant un projet de géothermie :
TEXTE: 
{}

TÂCHE:
Make a one page summary paying special attention to all the administrative matters like budgets, plans, actions to take in the future, organizational and hierarchical charts, announcements, meetings, contacts, elections, reports and all the related topics to these.

OUTPUT:
Publier directement le résumé.
"""

In [ ]:
model = "llama-3.2-70b-versatile"

messages = [
    {"role": "system",
            "content":  "Vous êtes un assistant utile" },
    {"role": "user",
            "content":  summary_prompt.format(text_to_sum)}
            ]

chat_completion = client.chat.completions.create(
    messages=messages,
    model=model,
)

print(chat_completion.choices[0].message.content)

In [108]:
def generate_summary_groq(base_summary_prompt, text_to_summarize, model):
        
        summary_prompt = base_summary_prompt.format(text_to_summarize)
        messages = [
                        {"role": "system",
                        "content":  "Vous êtes un assistant utile" },
                        {"role": "user",
                        "content":  summary_prompt}
                        ]
                        
        chat_completion = client.chat.completions.create(
                                                        messages=messages,
                                                        model=model,
                                                        )
        
        return chat_completion.choices[0].message.content


## INITIAL PREPROCESSING RECAP

So far we have the following pieces from the original text:
- the reconstructed text based on the paragraph clusters.
- the embeddings of the reconstructed clusters.
- the reconstructed paragraphs in ranges of 10 sentences.
- the embeddings of the paragraphs detached and squeezed.
- the cluster each paragraph belongs to.
- the indexes of the paragraphs of each cluster.
- the actual sentences split from SpaCy, in case we need them after.

We have the following augmentations done for the original text:
- Recognition of tables in pdf pages converted into images.
- Generation of explanations of those tables
- A set of models to generate summaries of text.